In [2]:
# Simplified version of https://github.com/huggingface/transformers/blob/master/examples/text-generation/run_generation.py

import logging

import numpy as np
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer



logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

device = torch.device("cuda")

def set_seed(seed, n_gpu):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length
        
        
def load_model(model_name_or_path = "gpt2",
               seed=42, 
               fp16 = False #Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit
              ):
    n_gpu = torch.cuda.device_count()
    logger.warning(
        "device: %s, n_gpu: %s, 16-bits training: %s",
        device,
        n_gpu,
        fp16,
    )
    set_seed(seed, n_gpu)

    # Initialize the model and tokenizer
    model_class, tokenizer_class = GPT2LMHeadModel, GPT2Tokenizer
    tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
    model = model_class.from_pretrained(model_name_or_path)
    model.to(device)

    if fp16:
        model.half()
        
    return model, tokenizer    
        

def generate_text(model,
                  tokenizer,
                  prompt = "",
                  length = 20,
                  num_return_sequences = 1,
                  k = 0,
                  p = 0.9,
                  temperature = 1.0,
                  stop_token = "<|endoftext|>",
                  repetition_penalty = 1.0,  # primarily useful for CTRL model; in that case, use 1.2
                 ):

    # The max length of the prompt is 1024 for gpt-2
    length = adjust_length_to_model(length, max_sequence_length=model.config.max_position_embeddings)

    # Different models need different input formatting and/or extra arguments
    # No preprocessing needed for gpt-2
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(device)

    if encoded_prompt.size()[-1] == 0:
        input_ids = None
    else:
        input_ids = encoded_prompt

    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=length + len(encoded_prompt[0]),
        temperature=temperature, #temperature of 1.0 has no effect, lower tend toward greedy sampling
        top_k=k,
        top_p=p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        num_return_sequences=num_return_sequences,
    ) # A list of tensors with ids

    # Remove the batch dimension when returning multiple sequences
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist() # A list of ids
        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]
        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)

    return generated_sequences

model, tokenizer = load_model()
texts = generate_text(model, tokenizer, prompt = "I like apples", length = 100, num_return_sequences = 5)

for text in texts:
    print("<<")
    print(text)

09/26/2020 22:31:04 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
09/26/2020 22:31:05 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /home/epd/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
09/26/2020 22:31:05 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /home/epd/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
09/26/2020 22:31:05 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /home/epd/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f28

<<
I like apples and oranges," says Jeridhez Buresil, M.D., clinical director of the California Center for Food Science and Research at El Chiquita Food Institute in Los Angeles. "The main things I've learned are that the pineapple really tastes better when I buy them."

Coffee — which includes almonds and coconut — has been linked to all kinds of ailments, including cancer, diabetes, heart problems, dementia and chronic arthritis.

In recent years, the market fo
<<
I like apples" to T.Rex. He has never had a serious crack.

"On top of all that, he's a dirty jock," said Howard of the political commentary he'll often produce.

Whether he's strong-armed, backed by his role as street talker and social activist, or extremely important in American politics, though, "stealthy" isn't a trait to be forgotten.

Trent he does the big things like doing a live TV appearance an
<<
I like apples in our pantry, they're very nice. I like apples that are a little bit too sweet," said Noland.

Noland sa